In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules

In [3]:
df=pd.read_csv('Segmentdata.csv')

In [4]:
df.head()

,Unnamed: 0,InvoiceDate,Invoice Time,CustomerID,InvoiceNo,StockCode,Description,Country,Quantity,UnitPrice,Revenue,Items availability,revenue_buckets,price_buckets,final_revenue
0,0,14.12.2017,6:00,AVpgMuGwLJeJML43KY_c,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom,6,2.55,15.30,In Stock,Very Good,Medium,20
1,1,08.09.2017,5:00,AVpgMuGwLJeJML43KY_c,536365,71053,WHITE METAL LANTERN,United Kingdom,6,3.39,20.34,In Stock,Excellent,High,30
2,2,10.10.2017,5:00,AVpgMuGwLJeJML43KY_c,536365,84406B,CREAM CUPID HEARTS COAT HANGER,United Kingdom,8,2.75,22.00,In Stock,Excellent,High,30
3,3,28.08.2017,7:00,AVpgMuGwLJeJML43KY_c,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,United Kingdom,6,3.39,20.34,In Stock,Excellent,High,30
4,4,24.10.2017,4:00,AVpgMuGwLJeJML43KY_c,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,United Kingdom,6,3.39,20.34,In Stock,Excellent,High,30


In [5]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19999 entries, 0 to 19998
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          19999 non-null  int64  
 1   InvoiceDate         19999 non-null  object 
 2   Invoice Time        19999 non-null  object 
 3   CustomerID          19999 non-null  object 
 4   InvoiceNo           19999 non-null  object 
 5   StockCode           19999 non-null  object 
 6   Description         19999 non-null  object 
 7   Country             19999 non-null  object 
 8   Quantity            19999 non-null  int64  
 9   UnitPrice           19999 non-null  float64
 10  Revenue             19999 non-null  float64
 11  Items availability  19999 non-null  object 
 12  revenue_buckets     19999 non-null  object 
 13  price_buckets       19999 non-null  object 
 14  final_revenue       19999 non-null  int64  
dtypes: float64(2), int64(3), object(10)
memory usage: 2.4

In [7]:
basket = (df[df['Country'] =="United Kingdom"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [8]:
basket

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,...,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0


In [9]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [19]:
frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.040349,(60 TEATIME FAIRY CAKE CASES)
1,0.035987,(72 SWEETHEART FAIRY CAKE CASES)
2,0.061069,(ALARM CLOCK BAKELIKE GREEN)
3,0.045802,(ALARM CLOCK BAKELIKE RED)
4,0.073064,(ASSORTED COLOUR BIRD ORNAMENT)
...,...,...
101,0.031625,"(LUNCH BAG SPACEBOY DESIGN, LUNCH BAG BLACK S..."
102,0.030534,"(PLEASE ONE PERSON METAL SIGN, WHITE HANGING H..."
103,0.038168,"(RED HANGING HEART T-LIGHT HOLDER, WHITE HANGI..."
104,0.039258,"(WHITE HANGING HEART T-LIGHT HOLDER, WOODEN PI..."


In [20]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(WHITE HANGING HEART T-LIGHT HOLDER),(ASSORTED COLOUR BIRD ORNAMENT),0.177754,0.073064,0.032715,0.184049,2.51900,0.019728,1.136019
1,(ASSORTED COLOUR BIRD ORNAMENT),(WHITE HANGING HEART T-LIGHT HOLDER),0.073064,0.177754,0.032715,0.447761,2.51900,0.019728,1.488933
2,(HEART OF WICKER SMALL),(HEART OF WICKER LARGE),0.058888,0.054526,0.030534,0.518519,9.50963,0.027323,1.963678
3,(HEART OF WICKER LARGE),(HEART OF WICKER SMALL),0.054526,0.058888,0.030534,0.560000,9.50963,0.027323,2.138892
4,(KNITTED UNION FLAG HOT WATER BOTTLE),(WHITE HANGING HEART T-LIGHT HOLDER),0.046892,0.177754,0.030534,0.651163,3.66329,0.022199,2.357107


In [22]:
rules[ (rules['lift'] >=5) &
       (rules['confidence'] >= 0.5) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(HEART OF WICKER SMALL),(HEART OF WICKER LARGE),0.058888,0.054526,0.030534,0.518519,9.509630,0.027323,1.963678
3,(HEART OF WICKER LARGE),(HEART OF WICKER SMALL),0.054526,0.058888,0.030534,0.560000,9.509630,0.027323,2.138892
7,(LUNCH BAG RED RETROSPOT),(LUNCH BAG BLACK SKULL.),0.052345,0.067612,0.031625,0.604167,8.935820,0.028086,2.355507
8,(LUNCH BAG SPACEBOY DESIGN),(LUNCH BAG BLACK SKULL.),0.042530,0.067612,0.031625,0.743590,10.997932,0.028749,3.636314
16,(WOODEN FRAME ANTIQUE WHITE),(WOODEN PICTURE FRAME WHITE FINISH),0.043621,0.062159,0.034896,0.800000,12.870175,0.032185,4.689204
17,(WOODEN PICTURE FRAME WHITE FINISH),(WOODEN FRAME ANTIQUE WHITE),0.062159,0.043621,0.034896,0.561404,12.870175,0.032185,2.180545


In [23]:
print("Total no.of HEART OF WICKER SMALL is "+ str(basket['HEART OF WICKER SMALL'].sum()))
print("Total no.of LUNCH BAG RED RETROSPOT is "+ str(basket['LUNCH BAG RED RETROSPOT'].sum()))
print("Total no.of WOODEN PICTURE FRAME WHITE FINISH is " + str(basket['WOODEN PICTURE FRAME WHITE FINISH'].sum()))

Total no.of HEART OF WICKER SMALL is 567.0
Total no.of LUNCH BAG RED RETROSPOT is 596.0
Total no.of WOODEN PICTURE FRAME WHITE FINISH is 306.0
